In [1]:
# Downloader
import pdb_numpy 
import requests
import concurrent.futures
import gzip
import os

def download_pdb(pdb_id):
    # Define the URL to download the PDB assembly file
    local_filename = f"{pdb_id}.pdb"
    if not os.path.exists(local_filename):
        url = f"https://files.rcsb.org/download/{pdb_id}.pdb"

        # Use the requests module to get the content of the URL
        response = requests.get(url)

        # Save the content of the response to the local filename
        with open(local_filename, "wb") as f:
            f.write(response.content)


        
    print(f"Downloaded {local_filename}")
    #try:
    pdb_line = PDB(local_filename)
    #except:
       # pdb_line = None
        #pass
    return pdb_line



# Define the PDB IDs and assembly numbers to download
pdb_ids = ["6LU7", "7K93", "5L73"]
"""
# Create a thread pool to download the PDB assemblies in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit each download task to the executor
    futures = []
    for pdb_id, assembly_num in zip(pdb_ids, assembly_nums):
        futures.append(executor.submit(download_pdb_assembly, pdb_id, "1"))

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)
"""
download_pdb("7K93")

ModuleNotFoundError: No module named 'pdb_numpy'

In [48]:
from pdb_numpy import Coor, Model
from pdb_numpy.format import mmcif, pdb

def cooredit(coor):
    try:
        local_rmsd = []
        ref = coor.select_atoms("protein")
        ref_seq_dict = ref.get_aa_seq()

        if len(ref_seq_dict) <= 6:
            ref = ref.add_symmetry()
            ref.compute_chains_CA()
            ref_seq_dict = ref.get_aa_seq()
            ref = ref.apply_transformation()
            ref.compute_chains_CA()
            ref_seq_dict = ref.get_aa_seq()
            ref = ref.remove_overlap_chain()
            ref_seq_dict = ref.get_aa_seq()

        if len(ref_seq_dict) <= 6:
            ref = ref.copy_box(x=2, y=2, z=2)
            ref.compute_chains_CA()
            ref = ref.remove_overlap_chain()
            ref_seq_dict = ref.get_aa_seq()
        return ref
    except:
        return coor

coords = Coor('7K93.pdb')
cooredit(coords)

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f PDB PDB("7K93.pdb")

In [ ]:
PDB("7K93.pdb")

In [ ]:
df4 = pd.DataFrame()
df4.iloc[0:0]
from tqdm import tqdm

with open('mehdi.pickle', 'rb') as f:
    mehdi_list = pickle.load(f)
    
for struct in tqdm(mehdi_list):
    #try:
    new_row = download_pdb(struct) 
    amyloid = {'is_amyloid':True}
    new_row2 = {**new_row,**amyloid}
    df4 = df4.append(new_row2, ignore_index=True)
    #except: 
     #   pass

with open('TESTSET_500.pickle', 'rb') as f:
    test_set = pickle.load(f)
    
for struct in tqdm(test_set):
    try:
        new_row = download_pdb(struct)
        amyloid = {'is_amyloid':False}
        new_row2 = {**new_row,**amyloid}
        df4 = df4.append(new_row2, ignore_index=True)
    except:
        pass


In [11]:
df4 = pd.DataFrame()
import copy
# Suppréssion des données dans le df
df4.iloc[0:0]

py

# Objectif: trouver les pdb dans test_set qui ne sont pas dans tempresults


0
0
76
500


In [ ]:
print(tempresults)

In [ ]:
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import logging
import time

print(len(df4))
time.sleep(5)
logging.getLogger("pdb_numpy").setLevel(logging.WARNING)
start_time = time.time()
def process_struct(struct, is_amyloid):
    #try:
    
    new_row = download_pdb(struct)
    if new_row != None:
        amyloid = {'is_amyloid': is_amyloid}
        new_row2 = {**new_row,**amyloid}
        processing_time = time.time() - start_time
        print(f"Processing of {struct} completed in {processing_time} seconds")
        print(f"{len(new_row2)}, PDB: {new_row2['PDB_ID']}")
        return new_row2
    else:
        return None
        
    #except:
        #print (str(e))
     #   return None

import time
start = time.time()
df4_list = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    # Process the `mehdi_list` items
    results = {executor.submit(process_struct, struct, True): struct for struct in mehdi_list}
    for future in tqdm(concurrent.futures.as_completed(results), total=len(results)):
        struct = results[future]
        try:
            new_row3 = future.result(timeout=60)
            if new_row3 is not None:
                df4_list.append(new_row3)
        except concurrent.futures.TimeoutError:
            future.cancel()
            print(f"Processing of {struct} timed out after 60 seconds")

    # Process the `test_set` items
    results = {executor.submit(process_struct, struct, False): struct for struct in test_set}
    for future in tqdm(concurrent.futures.as_completed(results), total=len(results)):
        struct = results[future]
        try:
            new_row4 = future.result(timeout=60)
            if new_row4 is not None:
                df4_list.append(new_row4)
        except concurrent.futures.TimeoutError:
            future.cancel()
            print(f"Processing of {struct} timed out after 60 seconds")
    
    print(len(df4))

df4 = pd.DataFrame.from_dict(df4_list)
end = time.time()
print(end - start)

In [53]:
import dtale
#dtale.show(df4)
with open('tempresults','wb') as f:
    pickle.dump(df4,f)

In [52]:
print(len(df4))

343


In [55]:
dtale.show(df4)

2023-05-09 14:16:47,956 - INFO     - Executing shutdown due to inactivity...
2023-05-09 14:16:47,963 - INFO     - Executing shutdown...
2023-05-09 14:16:47,964 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/meuret/anaconda3/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/meuret/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/meuret/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/meuret/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/meuret/anaconda3/lib/python3.10/site-packages/dtale/app.py", line 433, in shutdown
    shutdown_server()
  File "/home/meuret/anac

In [ ]:
df5 = pd.DataFrame()
df5['Seq'] = df4['AAseq']
print(df5)
with open('DF5_with_AA.pickle','wb') as f:
    pickle.dump(df5,f)

In [50]:
#### import pdb_numpy
import pandas as pd
import dtale 
from pdb_numpy.format import mmcif, pdb
from pdb_numpy import Coor, Model
import pdb_numpy.DSSP as DSSP
import numpy as np
import freesasa
#from rdkit import Chem
#from rdkit.Chem import GraphDescriptors
#from rdkit.Chem import Descriptors
from collections import Counter
import pickle
import time
def PDB(pdb_path):
    print(pdb_path)
    try:
        coords = Coor(pdb_path)
        test = coords.select_atoms("protein")
    except:
        return None
    
    sequenceAA = test.get_aa_seq()
    temp = {val: key for key, val in sequenceAA.items()}
    res = {val:key for key, val in temp.items()}

    # Filter for structures too big
    AAseq = ''.join(list(res.values())).replace('-','')
    if len(AAseq) > 200:
        print(f"CANCELLED {pdb_path}, {len(AAseq)} AA")
        return None
    #print(AAseq)
    test = cooredit(test)
    
    #coor.write_pdb('7ZH7_new.pdb')
    #print(test.transformation)
    
    # Calcul de la matrice de liaisons hydrogènes
    try:
        HBOND = DSSP.compute_Hbond_matrix(test).view()
        result_dssp = DSSP.compute_DSSP(test)
        #print(len(HBOND))

        df = pd.DataFrame(HBOND)

        # Calcul des noms de residus correspondants à chaque chaine
        result = pd.unique(["%d_%s" % t for t in zip(test.resid, test.chain)])
        #print(result)
        #print(df)
        df.columns = result
        df.index = result

        # Matrice Oui/non sur liaisons hydrogènes mais pour les résidus
        coord = np.where(df)
        coordinates = [(x,y) for x, y in zip(coord[0], coord[1])]
        #print(coordinates)
        names = pd.unique(test.chain)
        df2 = pd.DataFrame(0, index=names, columns=names)

        for coord in coordinates:
            i, j = coord
            #print(f"Coords: {coord}, Index: {df.index[i]}, Column: {df.columns[j]}")
            row = df.index[i][-1:]
            column = df.columns[j][-1:]
            #print(f"Row: {row}, Column = {column}")
            df2[column][row] += 1

        #d = dtale.show(df,notebook=True)
        #d = dtale.show(df2)
        #print(d._main_url)

        unique_combinations = []

        # Determine le nombre de fois ou il y a des liaisons (matrice qui n'est pas symmetrique)
        for i in names:
            for j in names:
                pair = [i,j]
                if i != j and (pair[::-1] not in unique_combinations) :
                    unique_combinations.append(pair)

        #print(unique_combinations)
        total = {}
        numberofh = 0
        for combinations in unique_combinations:
            total[f"{combinations[0]}_{combinations[1]}"] = int(df2[combinations[0]][combinations[1]])+int(df2[combinations[1]][combinations[0]])
            numberofh = numberofh + int(df2[combinations[0]][combinations[1]])+int(df2[combinations[1]][combinations[0]])

        numbertotalh = numberofh/len(pd.unique(test.uniq_resid))*100
        numberHchains = numberofh/(len(pd.unique(test.uniq_resid)) * len(pd.unique(test.chain)))
    except:
        return None
    #Ecriture du fichier PDB
    #pathpdb = f"DOWN/{pdb_code}.pdb"
    #test.write(pathpdb)
    
    #Calcul avec FreeSasa du SASA
    structure = freesasa.Structure(pdb_path)
    result = freesasa.calc(structure)
    print(result.totalArea())
    area_classes = freesasa.classifyResults(result, structure)
    #for key in area_classes:
        #print(area_classes[key])
    
    # Calcul du molwt et du logp
    #m = Chem.rdmolfiles.MolFromPDBFile(pathpdb)
    #mollogppdb = Descriptors.MolLogP(m)
    #molwtpdb = Descriptors.ExactMolWt(m)
    
    #print(result_dssp)
    #print(type(result_dssp))
    
    # Calcul de la frequence de chaque lettre avec DSSP
    df3=pd.DataFrame.from_dict(result_dssp[0],orient="index", columns=["DSSP"])
    conc = ""
    for item in result_dssp[0]:
        conc += result_dssp[0][item]
    conc = conc.replace(" ", "")
    #print(conc)
    DSSP_letters = ["G","H","I","T","E","B","S"]
    letter_freq = Counter(conc)

    
    for lettre in DSSP_letters:
        if lettre not in letter_freq:
            letter_freq[lettre] = 0
        else:
            letter_freq[lettre] = letter_freq[lettre]/len(conc)
            
    df4 = pd.DataFrame()
    
    #Alignement du DSSP
    try:
        total, alignement_total = alignement(result_dssp)
        total_hydrophobicity,total_hydropathy = hydro_calculations(result)
    except:
        return None
    
    descriptors = {'PDB_ID':pdb_path[:4], 'Number of H':numbertotalh, 'H number w chains':numberHchains, 'SASA':result.totalArea(), 'Alignement_total':total, 'Alignement_moyen': alignement_total, 'DF_DSSP':df3,'DF_H':df2, 'AAseq':AAseq, 'Hydrophobicity':total_hydrophobicity, 'Hydropathy':total_hydropathy}
    new_row = {**descriptors,**letter_freq}
    
    return new_row

PDB('7K93.pdb')

7K93.pdb
CANCELLED 7K93.pdb, 2244 AA


In [1]:
from Bio import Align
from Bio.Align import PairwiseAligner
import pdb_numpy.DSSP as DSSP
import pdb_numpy
import dtale
import numpy as np
import pickle
import pandas as pd


aligner = PairwiseAligner()
def alignement(result_dssp):
    count = []
    longueur = []
    chains_letters = []

    for chain in result_dssp[0]:
        chains_letters.append(chain)
        #print(result_dssp[0][chain])
        # E beta ladder
        # B beta bridge
        # S Bend 
        count.append(result_dssp[0][chain].count('E'))
        longueur.append(len(result_dssp[0][chain]))
    #print(len(chains_letters))
    
    alignements_results = []
    for value in range(1,len(chains_letters)):
        alignement = aligner.align(result_dssp[0][chains_letters[0]],result_dssp[0][chains_letters[value]])
        #print(alignement.score)
        bit_score = (alignement.score / len(result_dssp[0][chains_letters[0]])*100)
        #print(bit_score)
        alignements_results.append(bit_score)
                
    countnp = np.array(count)
    longueurnp = np.array(longueur)
    resultats_alignements_np = np.array(alignements_results)


    total = (np.sum(countnp)/np.sum(longueurnp))*100
    alignement_moyen = np.mean(resultats_alignements_np) 
    #print(total)  
    #print(alignement_moyen)

    return total, alignement_moyen

In [2]:


def hydro_calculations(freesasa_result):
    # Eisenberg scale 
    # source: https://resources.qiagenbioinformatics.com/manuals/clcgenomicsworkbench/650/Hydrophobicity_scales.html
    hydrophobicity_scale = {
        'ALA': 0.62,
        'CYS': 0.29,
        'ASP': -0.9,
        'GLU': -0.74,
        'PHE': 1.19,
        'GLY': 0.48,
        'HIS': -0.40,
        'ILE': 1.38,
        'LYS': -1.50,
        'LEU': 1.06,
        'MET': 0.64,
        'ASN': -0.78,
        'PRO': 0.12,
        'GLN': -0.85,
        'ARG': -2.53,
        'SER': -0.18,
        'THR': -0.05,
        'VAL': 1.08,
        'TRP': 0.81,
        'TYR': 0.26
    }

    #source: DOI:10.2174/1874464810902030171
    hydropathy_scale = {
        'ALA': 1.8,
        'CYS': 2.5,
        'ASP': -3.5,
        'GLU': -3.5,
        'PHE': 2.8,
        'GLY': -0.4,
        'HIS': -3.2,
        'ILE': 4.5,
        'LYS': -3.9,
        'LEU': 3.8,
        'MET': 1.9,
        'ASN': -3.5,
        'PRO': -1.6,
        'GLN': -3.5,
        'ARG': -4.5,
        'SER': -0.8,
        'THR': -0.7,
        'VAL': 4.2,
        'TRP': -0.9,
        'TYR': -1.3   
    }

    # Calculate SASA for entire structure
    structure_sasa = freesasa_result
    test = structure_sasa.residueAreas()
    dicthydrophobicity = {}
    dicthydropathy = {}
    for x in test:
        for y in test[x]:
            #print(test[x][y].residueType)
            #print(hydrophobicity_scale[test[x][y].residueType])
            # Calcul de l'hydrophobicité relative à chaque résidu, en fonction du SASA
            # doi:10.1016/s0021-9673(03)00182-1. ISSN 0021-9673. PMID 12877193, source wikipedia hydrophobicity scales
            #print(float(hydrophobicity_scale[test[x][y].residueType])*test[x][y].total)
            dicthydrophobicity[f'{x}_{y}'] = float(hydrophobicity_scale[test[x][y].residueType])*test[x][y].total
            dicthydropathy[f'{x}_{y}'] = hydropathy_scale[test[x][y].residueType]

    total_hydrophobicity = sum(dicthydrophobicity.values())
    total_hydropathy = sum(dicthydropathy.values())
    return total_hydrophobicity,total_hydropathy

structure = freesasa.Structure('5K2G.pdb')
result = freesasa.calc(structure)
print(hydro_calculations(result))

NameError: name 'freesasa' is not defined

In [ ]:
import seaborn as sns
sns.displot(df4['is_amyloid'])

In [ ]:
d = dtale.show(df4)
print(d._main_url)

In [ ]:
with open('DF4_MEHDI_RANDOM.pickle','wb') as f:
    pickle.dump(df4,f)

In [ ]:
df5 = df4.drop(['DF_DSSP', 'DF_H', 'PDB_ID'], axis=1)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df5.drop('is_amyloid', axis=1), df5['is_amyloid'], test_size=0.2, random_state=42)

In [ ]:
print(y_pred)
print(y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = lr.predict(X_test)
print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")

In [ ]:
print(X_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred = rf.predict(X_test)

# Calculate the accuracy score of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the random forest classifier: {accuracy:.3f}')

In [ ]:
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
for i in range(3):
    tree = rf.estimators_[i]
    dot_data = export_graphviz(tree,feature_names=X_train.columns,filled=True,max_depht=2,impurity=False,proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)

In [ ]:
print(amypro)

In [ ]:
with open('amypro.pickle', 'rb') as f:
    amypro = pickle.load(f)

df6 = pd.DataFrame()
for struct in amypro:
    try:
        new_row = PDB(struct) 
        df6 = df6.append(new_row, ignore_index=True)
    except: 
        pass

In [ ]:
with open('DF6_amypro_calculated.pickle','wb') as f:
    pickle.dump(df6,f)

In [ ]:
d = dtale.show(df6)
print(d._main_url)

In [ ]:
df7 = df6.drop(['DF_DSSP', 'DF_H', 'PDB_ID'], axis=1)
y_pred = rf.predict(df7)
print(type(y_pred))
print(len(y_pred))
print(len(df6))
df8 = df6.drop(['DF_DSSP', 'DF_H'], axis=1)
df8['is_amyloid'] = y_pred

In [ ]:
d = dtale.show(df8)
print(d._main_url)

In [ ]:
from Bio import Align
from Bio.Align import PairwiseAligner
def align():
    count = []
    longueur = []
    chains_letters = []
    print(result_dssp)

    for chain in result_dssp[0]:
        chains_letters.append(chain)
        #print(result_dssp[0][chain])
        # E beta ladder
        # B beta bridge
        # S Bend 
        count.append(result_dssp[0][chain].count('E'))
        longueur.append(len(result_dssp[0][chain]))
    #print(len(chains_letters))
    
    alignements_results = []
    for value in range(1,len(chains_letters)):
        alignement = aligner.align(result_dssp[0][chains_letters[0]],result_dssp[0][chains_letters[value]])
        #print(alignement.score)
        bit_score = (alignement.score / len(result_dssp[0][chains_letters[0]])*100)
        #print(bit_score)
        alignements_results.append(bit_score)
                
    countnp = np.array(count)
    longueurnp = np.array(longueur)
    resultats_alignements_np = np.array(alignements_results)


    total = (np.sum(countnp)/np.sum(longueurnp))*100
    alignement_moyen = np.mean(resultats_alignements_np)

In [ ]:
print(X_test)

In [ ]:
import nglview
view = nglview.show_file('7ZH7_new.pdb')
 ``

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df2, annot=True, cmap='coolwarm', ax=ax)
ax.set_xlabel('Acceptor Residue')
ax.set_ylabel('Donor Residue')
ax.set_title('H-Bond Contacts between Protein Chains')
plt.show()

In [ ]:
import inspect
lines = inspect.getsource(PDB)
print(lines)

In [ ]:
with open('results.pickle', 'rb') as f:
    res = pickle.load(f)

In [ ]:
print(res)